In [1]:
# Central object of ORM is registry.
from sqlalchemy.orm import registry

mapper_registry = registry()

In [2]:
from sqlalchemy import Column, Integer, String

# Use registry as a decorator.
@mapper_registry.mapped
class User:
    __tablename__ = "user_account"

    id = Column(Integer, primary_key=True)
    username = Column(String)
    fullname = Column(String)

    def __repr__(self):
        return "<User(%r, %r)>" % (self.username, self.fullname)

In [3]:
# the User class now has a Table object associated with it.
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('username', String(), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

In [5]:
User.__tablename__

'user_account'

In [4]:
User.__mapper__

<Mapper at 0x7fab84c3ec10; User>

In [7]:
from sqlalchemy import select

print(select(User.__table__))

SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account


In [8]:
spongebob = User(username="spongebob", fullname="Spongebob Squarepants")
spongebob

<User('spongebob', 'Spongebob Squarepants')>

In [12]:
repr(spongebob.id)

'None'

In [14]:
# Create a database schema.
from sqlalchemy import create_engine

engine = create_engine("sqlite://", echo=True)
with engine.begin() as connection:
    mapper_registry.metadata.create_all(connection)

2023-06-24 18:49:56,507 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 18:49:56,508 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-06-24 18:49:56,508 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-24 18:49:56,509 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2023-06-24 18:49:56,510 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-24 18:49:56,510 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	username VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2023-06-24 18:49:56,511 INFO sqlalchemy.engine.Engine [no key 0.00033s] ()
2023-06-24 18:49:56,511 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
# Session will handle the job of connecting, committing, and 
# releasing connections to this engine.
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine, future=True)

session = Session()

session.add(spongebob)

In [17]:
# The newly added entry will not be commited directly, which is a pending object.
session.new

IdentitySet([<User('spongebob', 'Spongebob Squarepants')>])

In [18]:
# Query the row will first autoflush the pending changes. Then return the query result.
from sqlalchemy import select

select_statement = select(User).filter_by(username="spongebob")
result = session.execute(select_statement)

2023-06-24 19:09:59,495 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 19:09:59,497 INFO sqlalchemy.engine.Engine INSERT INTO user_account (username, fullname) VALUES (?, ?)
2023-06-24 19:09:59,498 INFO sqlalchemy.engine.Engine [generated in 0.00076s] ('spongebob', 'Spongebob Squarepants')
2023-06-24 19:09:59,499 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account 
WHERE user_account.username = ?
2023-06-24 19:09:59,500 INFO sqlalchemy.engine.Engine [generated in 0.00034s] ('spongebob',)


In [19]:
# .scalar() will return the first column of each row.
also_spongebob = result.scalar()
also_spongebob

<User('spongebob', 'Spongebob Squarepants')>

In [20]:
# The row has been inserted so it has an id now.
spongebob.id

1

In [21]:
# The retrieved row is exactly the inserted object.
spongebob is also_spongebob

True

In [22]:
session.identity_map.items()

[((__main__.User, (1,), None), <User('spongebob', 'Spongebob Squarepants')>)]

In [23]:
session.add_all(
    [
        User(username="patrick", fullname="Patrick Star"),
        User(username="sandy", fullname="Sandy Cheeks"),
    ]
)

In [24]:
# The object will be marked as dirty.
spongebob.fullname = "Spongebob Jones"

In [27]:
print(session.dirty)
print(session.new)

IdentitySet([<User('spongebob', 'Spongebob Jones')>])
IdentitySet([<User('patrick', 'Patrick Star')>, <User('sandy', 'Sandy Cheeks')>])


In [31]:
# The transaction is committed, followed by a flush of the changes.
session.commit()

2023-06-24 19:30:09,675 INFO sqlalchemy.engine.Engine COMMIT


In [32]:
# After the session is commited,the session invalidates all the data aand remove the transaction.
# Access the old objects will need new queries.
spongebob.fullname

2023-06-24 19:30:12,037 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 19:30:12,038 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.username AS user_account_username, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2023-06-24 19:30:12,038 INFO sqlalchemy.engine.Engine [cached since 562s ago] (1,)


'Spongebob Jones'

In [33]:
# Roll back the transaction.
spongebob.username = "Spongy"
fake_user = User(username="fakeuser", fullname="Invalid")
session.add(fake_user)

result = session.execute(
    select(User).where(User.username.in_(["Spongy", "fakeuser"]))
)
result.all()

2023-06-24 19:38:34,360 INFO sqlalchemy.engine.Engine UPDATE user_account SET username=? WHERE user_account.id = ?
2023-06-24 19:38:34,361 INFO sqlalchemy.engine.Engine [generated in 0.00111s] ('Spongy', 1)
2023-06-24 19:38:34,362 INFO sqlalchemy.engine.Engine INSERT INTO user_account (username, fullname) VALUES (?, ?)
2023-06-24 19:38:34,362 INFO sqlalchemy.engine.Engine [cached since 1715s ago] ('fakeuser', 'Invalid')
2023-06-24 19:38:34,363 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account 
WHERE user_account.username IN (?, ?)
2023-06-24 19:38:34,364 INFO sqlalchemy.engine.Engine [generated in 0.00039s] ('Spongy', 'fakeuser')


[(<User('Spongy', 'Spongebob Jones')>,), (<User('fakeuser', 'Invalid')>,)]

In [34]:
session.rollback()

print(spongebob.username)
print(fake_user in session)     # Data is gone from the session.

2023-06-24 19:39:24,581 INFO sqlalchemy.engine.Engine ROLLBACK
2023-06-24 19:39:24,582 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 19:39:24,583 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.username AS user_account_username, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2023-06-24 19:39:24,583 INFO sqlalchemy.engine.Engine [cached since 1115s ago] (1,)
spongebob
False


In [37]:
# The data is also gone from the database.
result = session.execute(
    select(User).where(User.username.in_(["spongebob", "fakeuser"]))
)
result.all()

2023-06-24 19:41:01,554 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account 
WHERE user_account.username IN (?, ?)
2023-06-24 19:41:01,555 INFO sqlalchemy.engine.Engine [cached since 147.2s ago] ('spongebob', 'fakeuser')


[(<User('spongebob', 'Spongebob Jones')>,)]

In [38]:
print(User.username == "spongebob")

user_account.username = :username_1


In [39]:
query = (
    select(User).where(User.username == "spongebob").order_by(User.id)
)

# Return a tuple representing one row.
result = session.execute(query)
for row in result:
    print(row)

2023-06-24 19:56:19,752 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account 
WHERE user_account.username = ? ORDER BY user_account.id
2023-06-24 19:56:19,752 INFO sqlalchemy.engine.Engine [generated in 0.00095s] ('spongebob',)
(<User('spongebob', 'Spongebob Jones')>,)


In [40]:
result = session.execute(query)
for user_obj in result.scalars():
    print(user_obj)

2023-06-24 19:58:40,607 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account 
WHERE user_account.username = ? ORDER BY user_account.id
2023-06-24 19:58:40,608 INFO sqlalchemy.engine.Engine [cached since 140.9s ago] ('spongebob',)
<User('spongebob', 'Spongebob Jones')>


In [41]:
result = session.execute(query)
user_obj = result.scalars().one()
print(user_obj)

2023-06-24 20:08:38,843 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account 
WHERE user_account.username = ? ORDER BY user_account.id
2023-06-24 20:08:38,844 INFO sqlalchemy.engine.Engine [cached since 739.1s ago] ('spongebob',)
<User('spongebob', 'Spongebob Jones')>


In [42]:
query = select(User.username, User.fullname)
result = session.execute(query)
for row in result:
    print(f"{row.username} {row.fullname}")

2023-06-24 20:14:32,155 INFO sqlalchemy.engine.Engine SELECT user_account.username, user_account.fullname 
FROM user_account
2023-06-24 20:14:32,156 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ()
spongebob Spongebob Jones
patrick Patrick Star
sandy Sandy Cheeks


In [43]:
query = select(User, User.username)
result = session.execute(query)
for row in result:
    print(f"{row.User.id} {row.User.fullname} {row.username}")

2023-06-24 20:18:11,077 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname, user_account.username AS username__1 
FROM user_account
2023-06-24 20:18:11,078 INFO sqlalchemy.engine.Engine [generated in 0.00085s] ()
1 Spongebob Jones spongebob
2 Patrick Star patrick
3 Sandy Cheeks sandy


In [44]:
# filter_by is similar to where clause.
for (username, ) in session.execute(
    select(User.username).filter_by(
        fullname="Spongebob Jones"
    )
):
    print(username)

2023-06-24 20:18:31,734 INFO sqlalchemy.engine.Engine SELECT user_account.username 
FROM user_account 
WHERE user_account.fullname = ?
2023-06-24 20:18:31,734 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ('Spongebob Jones',)
spongebob


In [45]:
from sqlalchemy import or_

for (user, ) in (
    session.execute(
        select(User)
        .where(User.username == "spongebob")
        .where(or_(User.fullname == "Spongebob Jones", User.id < 5))
    )
):
    print(user)

2023-06-24 20:23:08,563 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account 
WHERE user_account.username = ? AND (user_account.fullname = ? OR user_account.id < ?)
2023-06-24 20:23:08,564 INFO sqlalchemy.engine.Engine [generated in 0.00134s] ('spongebob', 'Spongebob Jones', 5)
<User('spongebob', 'Spongebob Jones')>
